Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

NAME = "Oscar Delgado"
COLLABORATORS = "Collin Murphy"

---

In [ ]:
import $file.hw5stdlib
import hw5stdlib._

# Homework 5

In this assignment we will develop some helpful structures for building the next interpreter.


## Equality for $\mathbb{Z}$ and $\mathbb{B}$ (5 pts each)

Define equality for Integers and Booleans(It may be helpful to define equality for Natural numbers to use in your definition of Integer). These should be functions of the form:

$$
eq : A \rightarrow A \rightarrow \mathbb{B}
$$

Where you will want to fill each $A$ with the type of equality you are defining

In [ ]:
def nat_eq(x : Nat, y : Nat) : Bool = (x,y) match {
    case (Zero,Zero) => True
    case(x,Zero) => False
    case(Zero,y) => False
    case (Succ(x),Succ(y)) =>nat_eq(x,y)
}

In [ ]:
def int_eq(x : Integer, y : Integer) : Bool =(x,y) match {
    case (Positive(Zero),Negative(Zero)) => True
    case (Negative(Zero),Positive(Zero)) => True
    case (Positive(x),Negative(y)) => False
    case (Negative(x),Positive(y)) => False
    case (x,y) => nat_eq(abs(x),abs(y))
}

In [ ]:
// Autograder tests -- do not modify, duplicate, or delete
implicit def adtBool2nativeBool: Bool => Boolean = { case True => true ; case _ => false }
assert(int_eq(Positive(two), Positive(two)))
assert(!int_eq(Positive(two), Positive(five)))
assert(!int_eq(Positive(two), Negative(two)))
assert(int_eq(Negative(five),Negative(five)))
assert(int_eq(Positive(Zero),Negative(Zero)))

In [ ]:
def bool_eq(x : Bool, y : Bool) : Bool = (x,y) match{
    case (False,False) => True
    case (x,y) => and(x,y) 
}

In [ ]:
// Autograder tests -- do not modify, duplicate, or delete
assert(bool_eq(True,  True))
assert(bool_eq(False ,False))
assert(!bool_eq(True,  False))
assert(!bool_eq(False, True))

## Dictionaries

We will need to define the datatype $\texttt{Dictionary}$ before we write the first interpreter for lettuce. We'll begin by giving the definition below:

In [ ]:
sealed trait Dictionary[+A, +B] 
case object EmptyDict extends Dictionary[Nothing, Nothing]
case class Entry[A,B](key : A, value : B, xs : Dictionary[A, B]) extends Dictionary[A, B]

For both of the following functions you will need to provide a parameter for an equality-checking function. This is so we can check if keys are equal to eachother

### Membership(In) (5pts)

Write a function called `isIn` which returns `True` if the Dictionary contains a given key and `False` if it does not.

In [ ]:
def isIn[A,B](eq : ((A,A) => Bool), dict : Dictionary[A,B], k : A ) : Bool = (eq,dict,k) match{
    case (eq,EmptyDict,k) => False
    case (eq,Entry(a,b,xs),k)=> eq(a,k) match{
        case True => True
        case False => isIn(eq,xs,k)
    }
}

In [ ]:
// Autograder tests -- do not modify, duplicate, or delete
val test_dictionary : Dictionary[Nat, String] =
Entry(Zero, "Zero",
      Entry(one, "One",
            Entry(two,"Two",
                  EmptyDict)))
assert( isIn(nat_eq, test_dictionary, Zero))
assert( isIn(nat_eq, test_dictionary, one))
assert( isIn(nat_eq, test_dictionary, two))
assert(!isIn(nat_eq, test_dictionary, three))
assert(!isIn(nat_eq, test_dictionary, eight))

### Lookup (5 pts)

Write a function that retrieves a value that corresponds to a key in a dictionary. If the key is not in the map then return Nothing.

$$
\texttt{lookup} : (a \rightarrow a \rightarrow \mathbb{B}) \rightarrow \texttt{Dict}\ a\ b \rightarrow a \rightarrow \texttt{Maybe}\ b
$$

In [ ]:
def lookup[A,B](eq : (A,A) => Bool, dict : Dictionary[A,B], k : A) : Maybe[B] = (eq,dict,k) match{
    case (eq,EmptyDict,k) => Nothing
    case (eq,dict,Nothing) => Nothing
    case (eq,Entry(a,b,xs),k)=> eq(a,k) match{
        case True => Just(b)
        case False => lookup(eq,xs,k)
    }
}

In [ ]:
// Autograder tests -- do not modify, duplicate, or delete
assert(lookup(nat_eq, test_dictionary,Zero) equals Just("Zero"))
assert(lookup(nat_eq, test_dictionary,one) equals Just("One"))
assert(lookup(nat_eq, test_dictionary,two) equals Just("Two"))
assert(lookup(nat_eq, test_dictionary,three) equals Nothing)